# Different Class Different Color (DCDC) Text-Vision Comparison Test

This notebook tests model performance on distinguishing objects that differ in BOTH class AND color using text-vision zero-shot classification.

Test format:
- 4-way forced choice
- Query: Image with specific class and color (e.g., red apple)
- Text prompts: Include color + class (e.g., "red apple", "blue car", "green ball", "yellow book")
- Distractors: Different classes with different colors
- 4000 trials total per model

In [1]:
# Imports
import os
import sys
import pandas as pd
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import random
from datetime import datetime
import clip
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader

# Path setup - Use absolute paths to avoid any confusion
REPO_ROOT = r'C:\Users\jbats\Projects\NTU-Synthetic'

# Add discover-hidden-visual-concepts to path
DISCOVER_ROOT = os.path.join(REPO_ROOT, 'discover-hidden-visual-concepts')
sys.path.insert(0, DISCOVER_ROOT)
sys.path.insert(0, REPO_ROOT)

# Import from discover-hidden-visual-concepts repo
sys.path.append(os.path.join(DISCOVER_ROOT, 'src'))
from utils.model_loader import load_model
from models.feature_extractor import FeatureExtractor

# Paths - Updated for SyntheticKonkle
DATA_PATH = os.path.join(REPO_ROOT, 'data', 'SyntheticKonkle_224', 'SyntheticKonkle')
METADATA_PATH = os.path.join(REPO_ROOT, 'data', 'SyntheticKonkle', 'master_labels.csv')
RESULTS_PATH = os.path.join(REPO_ROOT, 'PatrickProject', 'Chart_Generation', 'text_vision_results.csv')

print(f"Data path: {DATA_PATH}")
print(f"Metadata path: {METADATA_PATH}")
print(f"Results will be saved to: {RESULTS_PATH}")

c:\Users\jbats\miniconda3\envs\ntu-synthetic\lib\site-packages\clip\clip.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging


Data path: C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle_224\SyntheticKonkle
Metadata path: C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\master_labels.csv
Results will be saved to: C:\Users\jbats\Projects\NTU-Synthetic\PatrickProject\Chart_Generation\text_vision_results.csv


In [2]:
# Load and prepare data
def load_synthetickonkle_data():
    """Load SyntheticKonkle dataset with metadata"""
    # Read metadata
    df = pd.read_csv(METADATA_PATH)
    
    # SyntheticKonkle has lowercase column names
    # Columns: folder, filename, class, color, size, texture, variant
    
    # Build full paths
    df['image_path'] = df.apply(lambda row: os.path.join(DATA_PATH, row['folder'], row['filename']), axis=1)
    
    # Filter to only entries with valid color information
    df = df[df['color'].notna() & (df['color'] != '')].copy()
    
    # Standardize color names (lowercase)
    df['color'] = df['color'].str.lower().str.strip()
    
    # Create class-color combination column
    df['class_color'] = df['class'] + '_' + df['color']
    
    print(f"Loaded {len(df)} images with color annotations")
    print(f"Unique classes: {df['class'].nunique()}")
    print(f"Unique colors: {df['color'].nunique()}")
    print(f"Unique class-color combinations: {df['class_color'].nunique()}")
    
    return df

# Load data
data_df = load_synthetickonkle_data()
print("\nSample data:")
print(data_df[['class', 'color', 'class_color']].head())

Loaded 7882 images with color annotations
Unique classes: 67
Unique colors: 12
Unique class-color combinations: 671

Sample data:
    class   color    class_color
0  abacus     red     abacus_red
1  abacus   green   abacus_green
2  abacus    blue    abacus_blue
3  abacus  yellow  abacus_yellow
4  abacus  orange  abacus_orange


In [3]:
def run_dcdc_text_vision_test(model_name='cvcl-resnext', seed=0, device=None, num_trials=4000):
    """Run Different Class Different Color text-vision test
    
    Args:
        model_name: Model to test ('cvcl-resnext' or 'clip-res')
        seed: Random seed for reproducibility (matches original Class test)
        device: Device to use (None for auto-detect)
        num_trials: Total number of trials to run
    """
    # Set seeds to match original test methodology
    random.seed(seed)
    torch.manual_seed(seed)
    
    print(f"\n{'='*60}")
    print(f"Running DCDC Text-Vision Test with {model_name}")
    print(f"{'='*60}")
    
    # Device selection
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    if device == 'cuda' and not torch.cuda.is_available():
        print("[ERROR] CUDA requested but not available! Falling back to CPU.")
        device = 'cpu'
    
    print(f"Using device: {device}")
    
    # Load model using same loader as original
    print(f"[INFO] Loading {model_name} on {device}...")
    model, transform = load_model(model_name, seed=seed, device=device)
    extractor = FeatureExtractor(model_name, model, device)
    model.eval()
    
    # Load and prepare data
    df = pd.read_csv(METADATA_PATH)
    
    # Build full paths
    df['image_path'] = df.apply(lambda row: os.path.join(DATA_PATH, row['folder'], row['filename']), axis=1)
    
    # Filter to only entries with valid color information
    df = df[df['color'].notna() & (df['color'] != '')].copy()
    
    # Standardize color names (lowercase)
    df['color'] = df['color'].str.lower().str.strip()
    
    # Create class-color combination column
    df['class_color'] = df['class'] + '_' + df['color']
    
    print(f"Loaded {len(df)} images with color annotations")
    print(f"Unique classes: {df['class'].nunique()}")
    print(f"Unique colors: {df['color'].nunique()}")
    print(f"Unique class-color combinations: {df['class_color'].nunique()}")
    
    # Group data by class-color combinations
    grouped = df.groupby('class_color').agg({
        'image_path': list,
        'class': 'first',
        'color': 'first'
    }).reset_index()
    
    # Filter to combinations with at least 1 image
    grouped = grouped[grouped['image_path'].apply(len) >= 1].reset_index(drop=True)
    
    print(f"\nUsing {len(grouped)} unique class-color combinations")
    
    # Pre-compute all image embeddings for efficiency
    print("\nExtracting image embeddings...")
    image_embeddings = {}
    skipped_images = []
    
    # Process in batches
    all_image_paths = [img for imgs in grouped['image_path'] for img in imgs]
    batch_size = 16
    
    for i in tqdm(range(0, len(all_image_paths), batch_size), desc="Extracting embeddings"):
        batch_paths = all_image_paths[i:i+batch_size]
        batch_images = []
        
        for img_path in batch_paths:
            if img_path not in image_embeddings:  # Skip if already processed
                try:
                    img = Image.open(img_path).convert('RGB')
                    img_processed = transform(img).unsqueeze(0).to(device)
                    batch_images.append((img_path, img_processed))
                except Exception as e:
                    # Skip corrupted/invalid images
                    skipped_images.append(img_path)
                    continue
        
        if batch_images:
            # Stack batch
            paths = [p for p, _ in batch_images]
            imgs = torch.cat([img for _, img in batch_images], dim=0)
            
            # Get embeddings
            with torch.no_grad():
                embeddings = extractor.get_img_feature(imgs)
                embeddings = extractor.norm_features(embeddings)
            
            # Store
            for path, emb in zip(paths, embeddings):
                image_embeddings[path] = emb.cpu().float()  # Ensure float32
    
    print(f"Extracted embeddings for {len(image_embeddings)} images")
    if skipped_images:
        print(f"Skipped {len(skipped_images)} corrupted/invalid images")
    
    # Update grouped data to only include valid images
    for idx, row in grouped.iterrows():
        valid_paths = [p for p in row['image_path'] if p in image_embeddings]
        grouped.at[idx, 'image_path'] = valid_paths
    
    # Filter out combinations with no valid images
    grouped = grouped[grouped['image_path'].apply(len) > 0].reset_index(drop=True)
    
    # Prepare for trials
    correct_count = 0
    trial_results = []
    
    # Calculate trials per combination (ensure we get exactly num_trials)
    combinations_list = grouped['class_color'].tolist()
    trials_per_combo = num_trials // len(combinations_list)
    remaining_trials = num_trials % len(combinations_list)
    
    # Create trial distribution
    trial_distribution = []
    for i, combo in enumerate(combinations_list):
        n_trials = trials_per_combo + (1 if i < remaining_trials else 0)
        trial_distribution.extend([combo] * n_trials)
    
    # Shuffle trials
    random.shuffle(trial_distribution)
    
    print(f"\nRunning {len(trial_distribution)} trials...")
    
    # Run trials
    for trial_idx in tqdm(range(len(trial_distribution)), desc="Trials"):
        # Get query class-color
        query_combo = trial_distribution[trial_idx]
        query_data = grouped[grouped['class_color'] == query_combo].iloc[0]
        query_class = query_data['class']
        query_color = query_data['color']
        
        # Select random query image from valid images
        valid_query_paths = [p for p in query_data['image_path'] if p in image_embeddings]
        if not valid_query_paths:
            continue
        query_img_path = random.choice(valid_query_paths)
        
        # Select 3 distractors (different class AND different color)
        valid_distractors = grouped[
            (grouped['class'] != query_class) & 
            (grouped['color'] != query_color)
        ]
        
        # Filter distractors to those with valid images
        valid_distractors = valid_distractors[valid_distractors['image_path'].apply(len) > 0]
        
        if len(valid_distractors) < 3:
            continue  # Skip if not enough valid distractors
        
        distractor_combos = valid_distractors.sample(n=3)['class_color'].tolist()
        
        # Create candidate list (query + distractors)
        all_combos = [query_combo] + distractor_combos
        random.shuffle(all_combos)
        
        # Get correct index
        correct_idx = all_combos.index(query_combo)
        
        # Select random images for each candidate
        candidate_imgs = []
        candidate_texts = []
        skip_trial = False
        
        for combo in all_combos:
            combo_data = grouped[grouped['class_color'] == combo].iloc[0]
            valid_paths = [p for p in combo_data['image_path'] if p in image_embeddings]
            if not valid_paths:
                skip_trial = True
                break
            img_path = random.choice(valid_paths)
            candidate_imgs.append(img_path)
            
            # Create text prompt with color + class
            text_prompt = f"{combo_data['color']} {combo_data['class'].lower()}"
            candidate_texts.append(text_prompt)
        
        if skip_trial:
            continue
        
        # Encode text prompts (batch encoding)
        with torch.no_grad():
            if "clip" in model_name:
                tokens = clip.tokenize(candidate_texts, truncate=True).to(device)
                txt_features = model.encode_text(tokens)
                txt_features = extractor.norm_features(txt_features)
            else:  # CVCL
                tokens, token_len = model.tokenize(candidate_texts)
                tokens = tokens.to(device)
                if isinstance(token_len, torch.Tensor):
                    token_len = token_len.to(device)
                txt_features = model.encode_text(tokens, token_len)
                txt_features = extractor.norm_features(txt_features)
        
        # Get query image embedding
        query_embedding = image_embeddings[query_img_path].unsqueeze(0).to(device)
        
        # Calculate similarity following original predict() method
        # Ensure both are float32
        query_embedding = query_embedding.float()
        txt_features = txt_features.float()
        
        similarity = (100.0 * query_embedding @ txt_features.transpose(-2, -1)).softmax(dim=1)
        
        # Get prediction
        pred_idx = similarity.argmax(dim=1).item()
        
        # Check if correct
        is_correct = (pred_idx == correct_idx)
        if is_correct:
            correct_count += 1
        
        # Store trial result
        trial_results.append({
            'trial': trial_idx + 1,
            'query_class': query_class,
            'query_color': query_color,
            'query_img': os.path.basename(query_img_path),
            'correct_idx': correct_idx,
            'predicted_idx': pred_idx,
            'correct': is_correct,
            'candidate_texts': candidate_texts,
            'similarity_scores': similarity.cpu().numpy().tolist()
        })
    
    # Calculate accuracy
    accuracy = correct_count / len(trial_results) if trial_results else 0
    
    print(f"\n{'='*60}")
    print(f"Results for {model_name} - DCDC Text-Vision Test:")
    print(f"Total trials: {len(trial_results)}")
    print(f"Correct: {correct_count}")
    print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"{'='*60}")
    
    # Save results to CSV
    results_row = {
        'Model': model_name,
        'Test': 'DCDC-TextVision',
        'Dataset': 'SyntheticKonkle',  # Changed from KonkLab
        'Correct': correct_count,
        'Trials': len(trial_results),
        'Accuracy': accuracy
    }
    
    # Append to results file
    os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)
    if os.path.exists(RESULTS_PATH):
        results_df = pd.read_csv(RESULTS_PATH)
    else:
        results_df = pd.DataFrame()
    
    results_df = pd.concat([results_df, pd.DataFrame([results_row])], ignore_index=True)
    results_df.to_csv(RESULTS_PATH, index=False, float_format='%.4f')
    print(f"\nResults saved to {RESULTS_PATH}")
    
    return trial_results, accuracy

## Run CVCL DCDC Text-Vision Test

In [4]:
# Run CVCL test with seed=0 (matching original Class test)
cvcl_trials, cvcl_accuracy = run_dcdc_text_vision_test('cvcl-resnext', seed=0, num_trials=4000)


Running DCDC Text-Vision Test with cvcl-resnext
Using device: cuda
[INFO] Loading cvcl-resnext on cuda...
Loading checkpoint from C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt


Lightning automatically upgraded your loaded checkpoint from v1.5.8 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt`


Loaded 7882 images with color annotations
Unique classes: 67
Unique colors: 12
Unique class-color combinations: 671

Using 671 unique class-color combinations

Extracting image embeddings...


Extracting embeddings: 100%|██████████| 493/493 [00:31<00:00, 15.62it/s]


Extracted embeddings for 7841 images
Skipped 32 corrupted/invalid images

Running 4000 trials...


Trials: 100%|██████████| 4000/4000 [00:45<00:00, 88.51it/s]


Results for cvcl-resnext - DCDC Text-Vision Test:
Total trials: 4000
Correct: 1126
Accuracy: 0.2815 (28.15%)

Results saved to C:\Users\jbats\Projects\NTU-Synthetic\PatrickProject\Chart_Generation\text_vision_results.csv


## Run CLIP DCDC Text-Vision Test

In [5]:
# Run CLIP test with seed=0 (matching original Class test)
clip_trials, clip_accuracy = run_dcdc_text_vision_test('clip-resnext', seed=0, num_trials=4000)


Running DCDC Text-Vision Test with clip-resnext
Using device: cuda
[INFO] Loading clip-resnext on cuda...
Loaded 7882 images with color annotations
Unique classes: 67
Unique colors: 12
Unique class-color combinations: 671

Using 671 unique class-color combinations

Extracting image embeddings...


Extracting embeddings:   0%|          | 0/493 [00:00<?, ?it/s]c:\Users\jbats\miniconda3\envs\ntu-synthetic\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Extracting embeddings: 100%|██████████| 493/493 [00:22<00:00, 21.47it/s]


Extracted embeddings for 7841 images
Skipped 32 corrupted/invalid images

Running 4000 trials...


Trials: 100%|██████████| 4000/4000 [00:26<00:00, 151.15it/s]


Results for clip-resnext - DCDC Text-Vision Test:
Total trials: 4000
Correct: 3949
Accuracy: 0.9872 (98.72%)

Results saved to C:\Users\jbats\Projects\NTU-Synthetic\PatrickProject\Chart_Generation\text_vision_results.csv


## Compare Results

In [ ]:
# Display comparison
print("\n" + "="*60)
print("DCDC TEXT-VISION TEST COMPARISON")
print("="*60)
print(f"\nTest: Different Class Different Color (4-way forced choice)")
print(f"Text format: '{color} {class}' (e.g., 'red apple')")
print(f"\nResults:")
print(f"  CVCL Accuracy: {cvcl_accuracy:.4f} ({cvcl_accuracy*100:.2f}%)")
print(f"  CLIP Accuracy: {clip_accuracy:.4f} ({clip_accuracy*100:.2f}%)")
print(f"\nDifference: {abs(cvcl_accuracy - clip_accuracy):.4f} ({abs(cvcl_accuracy - clip_accuracy)*100:.2f}%)")
if cvcl_accuracy > clip_accuracy:
    print(f"CVCL performs better by {(cvcl_accuracy - clip_accuracy)*100:.2f}%")
elif clip_accuracy > cvcl_accuracy:
    print(f"CLIP performs better by {(clip_accuracy - cvcl_accuracy)*100:.2f}%")
else:
    print("Both models perform equally")

print("\n" + "="*60)

## Analysis Notes

### DCDC Text-Vision Test Characteristics:
- Tests discrimination when both class AND color differ
- Text prompts include color information (e.g., "red apple")
- Generally easier than SCDC since both features provide discrimination cues
- Models can leverage both visual and textual color understanding
